# 基于 LangGraph 和 LangChain 的长时记忆聊天 Agent

本项目实现一个具有长期记忆功能的聊天机器人，通过结合 LangGraph 的流程控制和 LangChain 的记忆组件，使机器人能够记住对话历史并利用它来提供上下文相关的回复。

## 主要功能

1. **长期记忆**：使用向量存储持久化对话记忆，超出即时上下文窗口的对话也能被检索
2. **记忆摘要**：定期对长对话生成摘要以保存关键信息
3. **工具使用**：集成了几个实用工具供 Agent 使用
4. **状态管理**：使用 LangGraph 管理对话状态和流程

## 实现技术

- **LangGraph**：用于构建 Agent 的状态图和流程控制
- **LangChain**：提供记忆组件、LLM 接口和工具集成
- **Chroma**：用作向量存储的数据库

# 带长时记忆的 Agent 聊天机器人

使用 LangChain 和 LangGraph 实现一个具有长期记忆能力的聊天机器人 Agent，能够记住之前的对话内容并在新的对话中引用这些信息。

## 安装和导入必要的库

首先，我们需要安装并导入 LangChain 和 LangGraph 相关的库。

In [ ]:
# 安装必要的库
!pip install langchain langchain-openai langgraph langchain-community 
!pip install chromadb langchain-core

In [ ]:
# 导入必要的库
import os
from typing import TypedDict, List, Dict, Any, Optional, Annotated
from datetime import datetime

# LangChain 相关导入
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# LangGraph 相关导入
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import MemoryContainer

## 初始化 LangChain 和 LangGraph

配置 LangChain API 密钥并设置模型参数。

In [ ]:
# 配置 OpenAI API 密钥
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"  # 请替换为你的 API 密钥

# 初始化 ChatOpenAI 模型
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# 创建系统提示信息
system_prompt = """你是一个友好的助手，拥有长期记忆能力。你能记住用户之前告诉你的信息，
并在回答时引用这些信息。当用户询问你之前提到过的信息时，请尝试从你的长期记忆中检索相关内容。
请保持礼貌、友好的态度，并尽可能提供有帮助的回答。"""

## 定义长时记忆存储

我们将使用向量数据库来实现长时记忆功能，这样聊天机器人可以存储和检索之前的对话内容。

In [ ]:
# 设置向量存储作为长期记忆
embedding_model = OpenAIEmbeddings()
vectorstore = Chroma(
    collection_name="memory_agent_conversations",
    embedding_function=embedding_model,
    persist_directory="./memory_agent_db"
)

# 定义状态类型
class AgentState(TypedDict):
    messages: List[BaseMessage]  # 当前对话消息历史
    long_term_memory: Dict[str, Any]  # 长期记忆存储
    current_time: str  # 当前时间，用于记录对话时间

In [ ]:
# 长期记忆检索函数
def retrieve_from_memory(query: str, k: int = 3):
    """从长期记忆中检索相关内容"""
    results = vectorstore.similarity_search(query, k=k)
    return [doc.page_content for doc in results]

# 存储到长期记忆的函数
def store_in_memory(text: str, metadata: Dict[str, Any] = None):
    """将文本存储到长期记忆中"""
    metadata = metadata or {}
    metadata["timestamp"] = datetime.now().isoformat()
    vectorstore.add_texts([text], metadatas=[metadata])

## 创建聊天机器人 Agent

现在，我们将使用 LangGraph 来创建一个支持长时记忆的聊天机器人 Agent。流程包括：接收用户输入、检索相关记忆、处理对话、生成回复并更新记忆。

In [ ]:
# 初始化 Agent 节点函数

def initialize_state() -> AgentState:
    """初始化代理状态"""
    return {
        "messages": [SystemMessage(content=system_prompt)],
        "long_term_memory": {},
        "current_time": datetime.now().isoformat()
    }

def receive_input(state: AgentState, input_message: str) -> AgentState:
    """接收用户输入并更新状态"""
    state["messages"].append(HumanMessage(content=input_message))
    state["current_time"] = datetime.now().isoformat()
    return state

def retrieve_memory(state: AgentState) -> AgentState:
    """检索相关记忆"""
    if len(state["messages"]) > 1:  # 确保有用户消息
        user_message = state["messages"][-1].content
        retrieved_memories = retrieve_from_memory(user_message)
        
        if retrieved_memories:
            memory_prompt = "\n\n相关历史记忆:\n" + "\n---\n".join(retrieved_memories)
            state["long_term_memory"]["retrieved"] = memory_prompt
    
    return state

def generate_response(state: AgentState) -> AgentState:
    """生成助手回复"""
    # 准备提示
    messages = state["messages"].copy()
    
    # 如果有检索到的记忆，添加到提示中
    if "retrieved" in state["long_term_memory"]:
        context_message = SystemMessage(
            content=f"以下是用户之前提到的相关信息，可以帮助你回答:\n{state['long_term_memory']['retrieved']}"
        )
        messages.insert(1, context_message)
    
    # 使用 LLM 生成回复
    response = llm.invoke(messages)
    state["messages"].append(response)
    
    # 保存对话到长期记忆
    dialogue_to_store = f"用户: {state['messages'][-2].content}\n助手: {response.content}"
    store_in_memory(dialogue_to_store, {"time": state["current_time"]})
    
    return state

In [ ]:
# 创建 LangGraph 工作流
workflow = StateGraph(AgentState)

# 添加节点
workflow.add_node("receive_input", receive_input)
workflow.add_node("retrieve_memory", retrieve_memory)
workflow.add_node("generate_response", generate_response)

# 定义边和工作流
workflow.set_entry_point("receive_input")
workflow.add_edge("receive_input", "retrieve_memory")
workflow.add_edge("retrieve_memory", "generate_response")
workflow.add_edge("generate_response", END)

# 编译工作流
memory_agent = workflow.compile()

## 与聊天机器人交互

现在我们可以与聊天机器人进行对话，并测试其长时记忆功能。我们将进行多轮对话，并检查机器人是否能记住之前的信息。

In [ ]:
# 初始化聊天状态
chat_state = initialize_state()

# 与聊天机器人交互的函数
def chat_with_agent(message: str) -> str:
    global chat_state
    
    # 运行工作流
    result = memory_agent.invoke({
        "state": chat_state,
        "input_message": message
    })
    
    # 更新状态
    chat_state = result
    
    # 返回最新的助手回复
    return chat_state["messages"][-1].content

# 测试对话
print("助手: 你好！我是一个具有长期记忆功能的助手。请问有什么我可以帮助你的？")

# 第一轮对话
user_input = "我叫小明，我喜欢读科幻小说"
print(f"用户: {user_input}")
response = chat_with_agent(user_input)
print(f"助手: {response}")

# 第二轮对话
user_input = "我还喜欢打篮球，每周会打三次球"
print(f"用户: {user_input}")
response = chat_with_agent(user_input)
print(f"助手: {response}")

# 第三轮对话，测试长期记忆
user_input = "你还记得我的名字吗？"
print(f"用户: {user_input}")
response = chat_with_agent(user_input)
print(f"助手: {response}")

# 第四轮对话，测试长期记忆中的爱好信息
user_input = "我都有哪些爱好？"
print(f"用户: {user_input}")
response = chat_with_agent(user_input)
print(f"助手: {response}")

## 功能扩展

我们可以进一步扩展聊天机器人的功能，例如添加记忆摘要、情感分析或用户偏好学习等。以下是一些可能的扩展方向：

1. **记忆摘要**: 定期总结过去的对话，创建更紧凑的记忆表示
2. **情感分析**: 分析用户情绪并相应调整回复风格
3. **用户偏好学习**: 识别并记住用户的偏好、兴趣和习惯
4. **对话主题追踪**: 识别对话主题的变化并据此检索相关记忆

In [ ]:
# 简单的记忆摘要功能示例

def summarize_conversation(messages: List[BaseMessage]) -> str:
    """总结对话内容"""
    # 提取用户和助手的消息
    conversation_text = "\n".join([
        f"{'用户' if isinstance(msg, HumanMessage) else '助手'}: {msg.content}"
        for msg in messages if not isinstance(msg, SystemMessage)
    ])
    
    # 创建摘要提示
    summary_prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content="你是一个对话摘要专家。请总结以下对话中的关键信息，特别是用户的个人信息、偏好和重要细节。"),
        HumanMessage(content=f"请总结以下对话:\n\n{conversation_text}")
    ])
    
    # 生成摘要
    chain = summary_prompt | llm | StrOutputParser()
    summary = chain.invoke({})
    
    return summary

# 测试总结功能
if len(chat_state["messages"]) > 2:  # 确保有足够的消息可以总结
    summary = summarize_conversation(chat_state["messages"])
    print("\n对话摘要:")
    print(summary)
    
    # 将摘要存储到长期记忆
    store_in_memory(f"对话摘要: {summary}", {"type": "summary", "time": datetime.now().isoformat()})

## 总结

在这个笔记本中，我们成功实现了一个具有长期记忆功能的聊天机器人 Agent，使用 LangChain 和 LangGraph 作为核心框架。关键组件包括：

1. **向量存储**: 用于保存和检索对话历史
2. **LangGraph 工作流**: 处理用户输入、检索记忆和生成回复
3. **记忆检索机制**: 基于相似度搜索从长期记忆中检索相关内容
4. **对话管理**: 维护对话状态并更新长期记忆

这个聊天机器人能够记住用户之前提供的信息，并在后续对话中引用这些信息，提供更加个性化和连贯的交互体验。通过扩展，还可以添加更多功能，如记忆摘要、情感分析等，进一步提升用户体验。